# Langgraph router example

## Lib import

In [ ]:
import os
import logging
from typing import List, Literal, Annotated, Optional, Union
from typing_extensions import TypedDict
import chromadb
from chromadb.config import Settings
from pydantic import BaseModel, Field, validator

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableSerializable
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_huggingface import HuggingFaceEmbeddings

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver

## Init and API key

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# key
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# deepseek
deepseek_llm_model = "deepseek-chat"

# silicon
silicon_base_url =  "https://api.siliconflow.cn/v1"
silicon_llm_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# huggingface
huggingface_embed_model = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
logging.basicConfig(level=logging.INFO)

## LLM Init

In [ ]:
# init deepseek LLM model
llm_deepseek = ChatDeepSeek(
    model=deepseek_llm_model,
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    top_p=0.9,
    frequency_penalty=0.7,
    presence_penalty=0.5,
    max_retries=3
)

## Deepseek Question Router

In [ ]:
################################################################################
### Question Router
################################################################################
class QuestionRouter:
    """
    LLM Router.
    Route a user query to the most relevant datasource.
    """

    def __init__(
        self,
        llm: RunnableSerializable,
    ):
        self.schema = self._default_schema()
        self.system_prompt = self._default_system_prompt()
        self.chain = self._build_chain(llm)
    
    @staticmethod
    def _default_schema():
        class RouteSchema(BaseModel):
            datasource: Literal["web_search", "vectorstore"] = Field(
                default="web_search",
                description="The chosen route based on the question.",
                examples=["web_search", "vectorstore"],
            )
            
        return RouteSchema
    
    @staticmethod
    def _default_system_prompt():
        role = """
            \rRole: Route Expert
            \rTask: Routing a user question to a vectorstore or web search
            \r
            \rGuidelines:
            \rThe vectorstore contains documents related to Calerie Health and the healthcare products it produces.
            \rUse the vectorstore for questions on these topics. Otherwise, use web-search.
        """

        respond_request = """
            \rCurrent Scenario: Rapid Response Mode
        """

        response_format = """
            \rPlease return to the chosen route based on the question.
            \rNo explanations or additional text.
        """
    
        return f"{role}\n{respond_request}\n{response_format}"
    
    def _build_chain(self, llm: RunnableSerializable) -> RunnableSerializable:
        structured_llm = llm.with_structured_output(self.schema)
        
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            ("human", "User question: {question}")
        ])
        
        return prompt_template | structured_llm
    
    def invoke(
        self,
        question: str,
        **kwargs
    ) -> Union[str, dict]:
        inputs = {
            "question": question,
            **kwargs
        }
        return self.chain.invoke(inputs)

## Test Router

In [ ]:
question_router = QuestionRouter(llm=llm_deepseek)

question = "What are the ingredients in Alpha Hope?"
print(f"question: {question}")
print(f"route: {question_router.invoke({"question": question})}")
print("\n")

question = "What are the active ingredients that provide anti-aging benefits to the human body?"
print(f"question: {question}")
print(f"route: {question_router.invoke({"question": question})}")
print("\n")

question = "Please describe how exercise is good for my health?"
print(f"question: {question}")
print(f"route: {question_router.invoke({"question": question})}")